In [1]:
import yfinance            as yf
import pandas              as pd
import numpy               as np
from sklearn.linear_model  import Lasso
from sklearn.model_selection import train_test_split

np.random.seed(123)

BTC-USD

btc_raw = yf.Ticker("BTC-USD").history(period="30y")

btc = btc_raw['Close']
btc

<class 'ModuleNotFoundError'>: No module named 'yfinance'

In [2]:
features = pd.DataFrame(btc.copy())
features['SMA_10'] = features['Close'].rolling(10).mean()
features['SMA_50'] = features['Close'].rolling(50).mean()
features['SMA_100'] = features['Close'].rolling(100).mean()
features['STD_14'] = features['Close'].pct_change().rolling(14).std()
features['Vol_Mean_20'] = doge_raw['Volume'].rolling(20).std()
features['Fake_Macro'] = np.random.rand(len(btc), 1)
features['Ret'] = features['Close'].pct_change().dropna()
features = features.drop('Close',axis=1)
features = features.dropna()

<class 'NameError'>: name 'pd' is not defined

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features.drop('Ret',axis=1), features['Ret'], test_size=0.3, shuffle=False)
    
    
model = Lasso().fit(X_train, y_train)  # ideally we would use LassoCV but dont have time series CVs

forecasts = model.predict(X_test) #no extra params for demo case

In [ ]:
model.score(X_test,y_test) #not good
y_test.plot()
pd.DataFrame(forecasts, index = y_test.index).plot()

In [ ]:
pd.DataFrame(forecasts, index = y_test.index).describe()

Backtester

In [ ]:
def backtester(signals,price, tcost = 0.005):

        pos_val = np.zeros(np.shape(price))
        cash    = np.zeros(np.shape(price))
        cash[0] = 1

        for i,val in enumerate(price):

            if i == len(price)-1:
                break


            if signals[i] == 0:

                cash[i+1] = (pos_val[i] * val * (1-tcost)) + cash[i]
                pos_val[i+1] = 0

            elif signals[i] == 1:

                pos_val[i+1] = (cash[i] / val)*((1-tcost)) + pos_val[i]
                cash[i+1] = 0


        returns = [a*b for a,b in zip(pos_val,price)] + cash
        
        return pd.DataFrame(returns, index = price.index)
    
def win_rate(sigs, returns):
    
    """
    Signals : series
    returns : series
    must be same length
    """
    
    tps = []

    sigs = sigs[1:-1].values.ravel()

    rets = (returns.pct_change()).shift(1).dropna().values.ravel()

    for i,val in enumerate(sigs):

        if (sigs[i] == 1 and rets[i]>0):
            tps.append(1)

    win_rate = sum(tps)/len(sigs)
    return win_rate

In [ ]:
strat_ret = backtester(pd.Series(sigs),doge[y_test.index[0]:])

strat_ret.plot()

Not the greatest winrate, likely due to price increases